## Deps

In [4]:
!pip install onnxmltools
!pip install lightgbm

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Load data

In [5]:
import pandas as pd
import io
import boto3

def getAddress(filename):     
    return 's3://{}/{}'.format('mn-ml-1', filename)

schoolFrame = pd.read_csv(getAddress('grad_school.csv'))
#schoolFrame.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


## Prep

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

features = schoolFrame[['gre','gpa','rank']]
scaled = MinMaxScaler().fit_transform(features)
scaledFrame = pd.DataFrame(scaled, columns=['gre','gpa','rank']).join(schoolFrame['admit'])
#scaledFrame.head()

# split the data into training and test sets
train, test = train_test_split(scaledFrame, test_size=0.2)

## Classify

In [7]:
from sklearn.ensemble import RandomForestClassifier

# define our Decision Tree model
clf = RandomForestClassifier()

# "fit" our model
clf.fit(train[['gre','gpa','rank']], train['admit'])

# calculate our prediction on the test set
predictions = clf.predict(test[['gre','gpa','rank']])

# calculate our accuracy
acc = accuracy_score(test['admit'], predictions)
print('Accuracy: ', acc)

NameError: name 'accuracy_score' is not defined

## Tuning

In [8]:
from sklearn.metrics import accuracy_score

rfResults = []
estimators = range(1, 100)
for number in estimators:

    # define our Decision Tree model
    clf = RandomForestClassifier(number)

    # "fit" our model
    clf.fit(train[['gre','gpa','rank']], train['admit'])

    # calculate our prediction on the test set
    predictions = clf.predict(test[['gre','gpa','rank']])

    # calculate our accuracy
    acc = accuracy_score(test['admit'], predictions)
    rfResults.append(acc)

%matplotlib inline
rfFrame = pd.DataFrame({ 'accuracy': rfResults, 'estimators': estimators }, columns = ['accuracy', 'estimators'])
#rfFrame.plot(x='estimators', y='accuracy')
#rfFrame.loc[rfFrame['accuracy'].idxmax()]
bestRandomForest = rfFrame[rfFrame['accuracy'] == rfFrame['accuracy'].max()]
bestRandomForest

,accuracy,estimators
11,0.75,12
48,0.75,49


## ONNX Export

In [10]:
import onnxmltools
from onnxmltools import convert_sklearn
from onnxmltools.convert.common.data_types import FloatTensorType
import torch
from torch.autograd import Variable

clf = RandomForestClassifier(bestRandomForest['estimators'].values[0])
clf.fit(train[['gre','gpa','rank']], train['admit'])

# convert the model to ONNX
model_onnx = convert_sklearn(clf, 'tree-based binary classifier', [('input', FloatTensorType([1, 3]))])

# export the model
onnxmltools.utils.save_model(model_onnx, 'random_forest.onnx')

simple model: <class 'sklearn.ensemble.forest.RandomForestClassifier'> 
